In [22]:
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#import pyLDAvis.gensim
import gensim
import pickle 
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.gensim
import os
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/JaredJose/Roblox-Scraper/main/data/filteredParentingPosts.csv?token=GHSAT0AAAAAAB6VRXEYVGN4MQA7CSPLYH4IZALOEIA')

In [24]:
df.head()

,url,title,author,date,timestamp,text,subreddit,score,upvote ratio
0,https://www.reddit.com/r/Parenting/comments/zp...,Roblox is like heroin for kids,Ava_Gras77,2022-12-19,1.671421e+09,Anyone else feel like Roblox is a kid’s gatewa...,t5_2qhn3,400,0.90
1,https://www.reddit.com/r/Parenting/comments/yp...,Alternatives to Roblox,skysenfr,2022-11-08,1.667925e+09,"Hi! So, a recent thread about 'scamming' and b...",t5_2qhn3,2,1.00
2,https://www.reddit.com/r/Parenting/comments/wh...,Is Roblox appropriate for my 9 yo?,Legal_Philosophy669,2022-08-06,1.659808e+09,My 9yo has been playing Roblox on their tablet...,t5_2qhn3,26,0.75
3,https://www.reddit.com/r/Parenting/comments/zw...,MIL bought a smartphone with SIM card for our ...,Stunning-Paper-5050,2022-12-27,1.672138e+09,So my mother in law gave our 6yr daughter a sm...,t5_2qhn3,1557,0.96
4,https://www.reddit.com/r/Parenting/comments/xz...,Need some non-judgmental advice. My 6 y/o spen...,jazzeriah,2022-10-09,1.665290e+09,My 6 y/o who will be 7 in December spends hour...,t5_2qhn3,2,0.62


In [25]:
df['text']

0     Anyone else feel like Roblox is a kid’s gatewa...
1     Hi! So, a recent thread about 'scamming' and b...
2     My 9yo has been playing Roblox on their tablet...
3     So my mother in law gave our 6yr daughter a sm...
4     My 6 y/o who will be 7 in December spends hour...
                            ...                        
83    I overheard my 8 year old niece saying this as...
84    I have a screen obsessed 5 year old and I need...
85    I had always been under the impression that Ro...
86    Over the holidays we've done lots of walks, pa...
87    So my son loves playing army men and loves pre...
Name: text, Length: 88, dtype: object

In [26]:
df['text'].str.count('game').sum()

97

In [27]:
text = " ".join(df['text'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [28]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
#print(bi_grams_list)

In [29]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: year old with a frequency of 25


In [30]:
vectorizer = CountVectorizer(ngram_range = (2, 2))
X1=vectorizer.fit_transform(bi_grams_list1)
features = (vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)


sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                 term  rank
8348        year old  25.0
2283       feel like  21.0
4845         nt know  15.0
5402  playing roblox  14.0
1025           ca nt  14.0
5360     play roblox  13.0
6209     screen time  13.0


In [31]:
df_ques = df['text'].str.endswith('?')
df_no_na = df.dropna(subset=['text'])
df_true = df_no_na.loc[df_ques]
print(df_true['text'])

0     Anyone else feel like Roblox is a kid’s gatewa...
1     Hi! So, a recent thread about 'scamming' and b...
2     My 9yo has been playing Roblox on their tablet...
3     So my mother in law gave our 6yr daughter a sm...
4     My 6 y/o who will be 7 in December spends hour...
5     Hey all, we have a 9 and 5 year old and we use...
7     For background Grandma (66) has always undermi...
9     I’m looking for advice. My 8yo son received a ...
10    so my son’s (6M) father (25M) has my son prima...
15    Husband and I were talking and our almost 8 ye...
16    So my kid, (10f) plays Roblox on my old phone....
17    My daughter is 9, she is going through puberty...
22    well this is probably a bit more complex.\n\nf...
23    I’m not a regular member of this sub but I’m h...
28    A few months ago my husband got a tablet for m...
30    I have four boys; 5, 8, 10 and 14. Truth be to...
31    I don't like that the platform allows unregula...
34    TL/DR: Roblox is huge and they are experim

In [32]:
# removes all commas...
df['text_processed'] = df['text'].apply(lambda x: re.sub('[,.!?]', '', x) if isinstance(x, str) else x)
df['text_processed'] = df['text_processed'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df['text_processed'].head()

0    anyone else feel like roblox is a kid’s gatewa...
1    hi so a recent thread about 'scamming' and bad...
2    my 9yo has been playing roblox on their tablet...
3    so my mother in law gave our 6yr daughter a sm...
4    my 6 y/o who will be 7 in december spends hour...
Name: text_processed, dtype: object

In [33]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.text.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['anyone', 'else', 'feel', 'like', 'roblox', 'kid', 'gateway', 'drug', 'believe', 'negative', 'effect', 'game', 'center', 'kids', 'life', 'feel', 'like', 'need', 'send', 'poor', 'kid', 'rehab', 'worry', 'look', 'forward', 'drugs', 'future', 'anyone', 'experience', 'roblox']


In [34]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
#s.makedirs('./results/ldavis_prepared_10')
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared
pyLDAvis.display(LDAvis_prepared)

/home/zhz094/.local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
